In [1]:
from datasets import load_dataset, Audio, DatasetDict, concatenate_datasets, get_dataset_config_names
import pandas as pd
import json

In [2]:
minds_14 = load_dataset("PolyAI/minds14", 'en-US', trust_remote_code=True)

In [3]:
configs = get_dataset_config_names("PolyAI/minds14")
print(configs)

['cs-CZ', 'de-DE', 'en-AU', 'en-GB', 'en-US', 'es-ES', 'fr-FR', 'it-IT', 'ko-KR', 'nl-NL', 'pl-PL', 'pt-PT', 'ru-RU', 'zh-CN', 'all']


In [4]:
print(minds_14)

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 563
    })
})


In [8]:
train_data = minds_14['train']
first10row = train_data.select(range(10))

In [11]:
first_10_df = pd.DataFrame(first10row)

# Display the DataFrame
first_10_df

,path,audio,transcription,english_transcription,intent_class,lang_id
0,C:\Users\Rowjak\.cache\huggingface\datasets\do...,{'path': 'C:\Users\Rowjak\.cache\huggingface\d...,I would like to set up a joint account with my...,I would like to set up a joint account with my...,11,4
1,C:\Users\Rowjak\.cache\huggingface\datasets\do...,{'path': 'C:\Users\Rowjak\.cache\huggingface\d...,Henry County set up a joint account with my wi...,Henry County set up a joint account with my wi...,11,4
2,C:\Users\Rowjak\.cache\huggingface\datasets\do...,{'path': 'C:\Users\Rowjak\.cache\huggingface\d...,hi I'd like to set up a joint account with my ...,hi I'd like to set up a joint account with my ...,11,4
3,C:\Users\Rowjak\.cache\huggingface\datasets\do...,{'path': 'C:\Users\Rowjak\.cache\huggingface\d...,how do I start a joint account,how do I start a joint account,11,4
4,C:\Users\Rowjak\.cache\huggingface\datasets\do...,{'path': 'C:\Users\Rowjak\.cache\huggingface\d...,can you help me set up a joint bank account,can you help me set up a joint bank account,11,4
5,C:\Users\Rowjak\.cache\huggingface\datasets\do...,{'path': 'C:\Users\Rowjak\.cache\huggingface\d...,how to set up a joint account,how to set up a joint account,11,4
6,C:\Users\Rowjak\.cache\huggingface\datasets\do...,{'path': 'C:\Users\Rowjak\.cache\huggingface\d...,is like you open a joint account tell me about...,is like you open a joint account tell me about...,11,4
7,C:\Users\Rowjak\.cache\huggingface\datasets\do...,{'path': 'C:\Users\Rowjak\.cache\huggingface\d...,can I have an account with my sister I want to...,can I have an account with my sister I want to...,11,4
8,C:\Users\Rowjak\.cache\huggingface\datasets\do...,{'path': 'C:\Users\Rowjak\.cache\huggingface\d...,I need to find out if I probably set up a join...,I need to find out if I probably set up a join...,11,4
9,C:\Users\Rowjak\.cache\huggingface\datasets\do...,{'path': 'C:\Users\Rowjak\.cache\huggingface\d...,I am calling because I would like to set up a ...,I am calling because I would like to set up a ...,11,4


In [10]:
print(first10row[:])

{'path': ['C:\\Users\\Rowjak\\.cache\\huggingface\\datasets\\downloads\\extracted\\fb34205d6a0b0af1c4ce29917bda9f94445cec5eed0737825fe9e1f87be1bba6\\en-US~JOINT_ACCOUNT\\602ba55abb1e6d0fbce92065.wav', 'C:\\Users\\Rowjak\\.cache\\huggingface\\datasets\\downloads\\extracted\\fb34205d6a0b0af1c4ce29917bda9f94445cec5eed0737825fe9e1f87be1bba6\\en-US~JOINT_ACCOUNT\\602baf24bb1e6d0fbce922a7.wav', 'C:\\Users\\Rowjak\\.cache\\huggingface\\datasets\\downloads\\extracted\\fb34205d6a0b0af1c4ce29917bda9f94445cec5eed0737825fe9e1f87be1bba6\\en-US~JOINT_ACCOUNT\\602b9f97963e11ccd901cc32.wav', 'C:\\Users\\Rowjak\\.cache\\huggingface\\datasets\\downloads\\extracted\\fb34205d6a0b0af1c4ce29917bda9f94445cec5eed0737825fe9e1f87be1bba6\\en-US~JOINT_ACCOUNT\\602bacab5f67b421554f6488.wav', 'C:\\Users\\Rowjak\\.cache\\huggingface\\datasets\\downloads\\extracted\\fb34205d6a0b0af1c4ce29917bda9f94445cec5eed0737825fe9e1f87be1bba6\\en-US~JOINT_ACCOUNT\\602b9d4cbb1e6d0fbce91fa4.wav', 'C:\\Users\\Rowjak\\.cache\\hugging

In [54]:
# Install dependencies if not already installed
# !pip install transformers datasets librosa evaluate noisereduce

# Import necessary libraries
import torch
from datasets import load_dataset, DatasetDict
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Seq2SeqTrainer, Seq2SeqTrainingArguments
import numpy as np
import librosa
import noisereduce as nr  # Library for noise reduction
from sklearn.model_selection import train_test_split
import evaluate

# Load the dataset
dataset = minds_14

In [55]:
# Load Wav2Vec2 Processor and Model for CTC
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to(device)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [56]:


# Define label mapping (e.g., mapping each intent to an integer label)
intent_to_id = {intent: idx for idx, intent in enumerate(set(dataset['train']['intent_class']))}

def preprocess_function(batch):
    # Mendapatkan array audio dan sampling rate dari kolom 'audio'
    audio = batch["audio"]["array"]
    orig_sampling_rate = batch["audio"]["sampling_rate"]

    # Resample audio ke 16 kHz jika diperlukan
    if orig_sampling_rate != 16000:
        audio = librosa.resample(audio, orig_sr=orig_sampling_rate, target_sr=16000)
    
    # Noise reduction dan normalisasi amplitudo
    audio = nr.reduce_noise(y=audio, sr=16000)
    audio = librosa.util.normalize(audio)
    
    # Konversi audio ke input untuk Wav2Vec2 dengan sampling_rate yang jelas
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt", padding=True)
    batch["input_values"] = inputs.input_values[0]

    # Ubah intent_class ke label numerik
    batch["labels"] = torch.tensor(intent_to_id[batch["intent_class"]])
    return batch

# Map preprocess function to dataset
processed_dataset = dataset.map(preprocess_function)


Map:   0%|          | 0/563 [00:00<?, ? examples/s]

In [57]:
# Pisahkan dataset yang sudah diproses menjadi train (70%), val (15%), dan test (15%)
train_testval_data = processed_dataset["train"].train_test_split(test_size=0.3, seed=42)
train_data = train_testval_data["train"]
testval_data = train_testval_data["test"]

# Bagi lagi testval_data menjadi validation dan test
val_test_split = testval_data.train_test_split(test_size=0.5, seed=42)
val_data = val_test_split["train"]
test_data = val_test_split["test"]

# Buat DatasetDict baru dengan pembagian train, validation, dan test
final_dataset = DatasetDict({
    "train": train_data,
    "validation": val_data,
    "test": test_data
})

# Cek hasil pembagian
print(final_dataset)

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id', 'input_values', 'labels'],
        num_rows: 394
    })
    validation: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id', 'input_values', 'labels'],
        num_rows: 84
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id', 'input_values', 'labels'],
        num_rows: 85
    })
})
